In [3]:
'''importing required modules here'''
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
from price_parser import Price
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from time import sleep,time



class MetroShoes:
    
    '''this class is about to scrape the metroshoes website'''
    
    def __init__(self,url,headers):
        
        '''initilizing class attributes here'''
        
        self.url = url
        self.headers = headers
        self.product_urls = []
        self.product_list = []
        self.crawl()
        
    def get_division_links(self):
        
        ''' this method is to send the initial request to website
        and to return the division links'''
        
        response = requests.get(self.url,headers=self.headers)
        soup  = BeautifulSoup(response.text,'html.parser')
        division_links = [urljoin(
            self.url,li.a['href']) for li in soup.find_all(
            'li',{'id':re.compile('[A-z]-shoes\d{1,3}')}) ]
        return division_links
    
        
    def get_product_urls(self,division_links):
        
        '''using this function we are getting all the product urls 
        from all the pages of division links that we got from previous func'''
        
        # this function helps us to navigate to each of the pages
        def pagination(url):
            # since the page is dynamically loaded we are using selenium to paginate
            '''-------driver setup-------'''
            
            path = r'C:\Users\asmen\Downloads\chromedriver.exe'
            options = Options()
            options.headless =True
            service = Service(path)
            driver = webdriver.Chrome(service=service,options=options)
            driver.execute_cdp_cmd('Network.setExtraHTTPHeaders',{'headers':self.headers}) # adding headers
            driver.get(url) # gets url and walks us to the page
            driver.implicitly_wait(5) # watis for 5 sec before timeout error
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
            wait = WebDriverWait(driver,1000)
            print('wait for some time to reach the bottom of the page')
            
            products_count = re.search(r'\d+',driver.find_element(By.XPATH,"//span[@class='item_count']").text).group()
            wait.until(lambda x:len(x.find_elements(By.XPATH,'//div[@class="col-md-3 col-sm-6 col-xs-6 col-6 new-class"]'))==int(products_count))
            print('***DONE***')
            return driver.page_source

                
        # we getting the product urls here
        for url in division_links:
            print(url)
            html = pagination(url) # calling pagination function
            soup = BeautifulSoup(html,'html.parser')
            product_urls = [p_tag.previous_element['href'] for p_tag in soup.find_all('p',{'class':'brand_name'}) if p_tag]
            self.product_urls = self.product_urls+ [urljoin(self.url,url) for url in product_urls]
            print(len(product_urls),len(self.product_urls))
                
    def parse_details(self):
        
        '''this function is about to parse the product details like
        name , brand , price size , color.. and  so on '''
        
        # to get the currency symbol
        def get_symbol(price_string):
            symbol = Price.fromstring(price_string).currency
            return symbol
        
        # to get price value as float 
        def get_amount(price_string):
            amount  = Price.fromstring(price_string).amount_float
            return amount
        
        # this helps use to avoid to do same thing again.
        def get_product_details(pattern):
            text = soup.find('span',{
                    'class':'label font-medium'},string=re.compile(
                f'{pattern}',re.I)).parent.find_all('span')[1].text.strip()
            return text

        url_list = [url for url in set(self.product_urls)]
        for url in url_list:

            response= requests.get(url,headers=self.headers)
            soup = BeautifulSoup(response.text,'html.parser')
            product_urls = url
            try:
                brand = soup.find('span',class_='proudctName').previous_element.strip()
            except:
                brand = ''
            try:
                name = soup.find('span',class_='proudctName').text.strip()
            except:
                name = ''
            try:
                price = soup.find('div',class_='price').span.text.strip()
                symbol = get_symbol(price) #getting symbol of the currency
                price = get_amount(price) # getting float of the price  value
            except Exception as e:
                price = ''
                symbol  = ''
            try:
                colors = ','.join([re.search(r'\d-([A-z]+)-',link['href']).group(1) for link in soup.find(
                    'div',{'class':'size-buttons'}).find_all('a')])
            except:
                colors = ''
            try:
                sizes = [button.span.text.strip() for button in soup.find('div',{
                    'class':'size-box'}).find_all('label')]
                size_numbers = ','.join(sizes) 
                size_country = soup.find('div',class_='size-labels').span.text.strip().replace('(','').replace(')','')
                
            except:
                sizes = ''
                size_numbers = ''
                size_country = ''
            try:
                gender  = get_product_details('gender')                
            except:
                 
                gender = ''
            try:
                occasion =  get_product_details('occasion')
            except:
                occasion = ''
            try:
                category = get_product_details('type')
            except:
                category = ''
            try:
                heel_type = get_product_details('heel')
            except:
                heel_type = ''
                
            #updating dictionary
            dic = {'product_url':url,
                   'product_brand':brand,
                   'product_name':name.replace(gender,'').strip(),
                   'gender':gender,
                   'heel_type':heel_type,
                   'occasion':occasion,
                   'type':category,
                   'product_price':price,
                   'currency_symbol':symbol,
                   'product_sizes':size_numbers,
                   'size_country':size_country,
                   'product_colors':colors,
                   'product_price':price,
            }

            # appending all the products detail to list
            self.product_list.append(dic)
            all_data = '\n'.join([str(value) for value in dic.values()])
            print('--'*50)
            print(' '*200)
            print(f'{all_data}')
            print(dic)
            print(' '*200)
            print('--'*50)

    def save_csv(self):
        df = pd.DataFrame(self.product_list)
        csv = df.to_csv('metroshoes.csv',index=False)
        
    def crawl(self):
        
        ''' this function is about to call all the function and 
        to manage the crawling process'''
        
        division_links = self.get_division_links()
        product_urls = self.get_product_urls(division_links)
        self.parse_details()
        self.save_csv()
                

In [4]:
if __name__=='__main__':
    start_time = time()
    URL = 'https://www.metroshoes.com/'
    HEADERS = {
    'authority': 'www.metroshoes.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9,ta;q=0.8,hi;q=0.7',
    'cache-control': 'no-cache',
    'cookie': 'HttpOnly; metros_last_list_path=%2Fkids-shoes%2Fshoe-type%2Floafers.html',
    'dnt': '1',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}
    metroshoes = MetroShoes(URL,HEADERS)
    end_time = time()
    time_taken = end_time-start_time
    print(f'time taken for executing the script is : {time_taken/60}')

https://www.metroshoes.com/men-shoes.html
wait for some time to reach the bottom of the page
***DONE***
993 993
https://www.metroshoes.com/women-shoes.html
wait for some time to reach the bottom of the page
***DONE***
1409 2402
https://www.metroshoes.com/kids-shoes.html
wait for some time to reach the bottom of the page
***DONE***
194 2596
----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-40-41-silver-party-sandals.html
Metro
Silver Party Sandals
Women
Block
Party
Sandals
2690.0
Rs
3,4,5,6,7,8
UK
black,gold,silver
{'product_url': 'https://www.metroshoes.com/metro-40-41-silver-party-sandals.html', 'product_brand': 'Metro', 'product_name': 'Silver Party Sandals', 'gender': 'Women', 'heel_type': 'Block', 'occ

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-33-3009-grey-casual-sandals.html
Metro
Grey Casual Sandals
Women
Flats
Casual
Sandals
1990.0
Rs
3,4,5,6,7,8
UK
grey,pink,yellow
{'product_url': 'https://www.metroshoes.com/metro-33-3009-grey-casual-sandals.html', 'product_brand': 'Metro', 'product_name': 'Grey Casual Sandals', 'gender': 'Women', 'heel_type': 'Flats', 'occasion': 'Casual', 'type': 'Sandals', 'product_price': 1990.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'grey,pink,yellow'}
                                                                                                                                                                   

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-71-8533-black-casual-loafers.html
Metro
Black Casual Loafers
Men
Flats
Casual
Loafers
3490.0
Rs
6,7,8,9,10,11
UK
black
{'product_url': 'https://www.metroshoes.com/metro-71-8533-black-casual-loafers.html', 'product_brand': 'Metro', 'product_name': 'Black Casual Loafers', 'gender': 'Men', 'heel_type': 'Flats', 'occasion': 'Casual', 'type': 'Loafers', 'product_price': 3490.0, 'currency_symbol': 'Rs', 'product_sizes': '6,7,8,9,10,11', 'size_country': 'UK', 'product_colors': 'black'}
                                                                                                                                                                                     

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/doubleu-351-1503-blue-casual-flip-flops.html
Doubleu
Blue Casual Flip Flops
Women
Flats
Casual
Flip Flops
1119.0
Rs
3,4,5,6,7,8
UK
blue
{'product_url': 'https://www.metroshoes.com/doubleu-351-1503-blue-casual-flip-flops.html', 'product_brand': 'Doubleu', 'product_name': 'Blue Casual Flip Flops', 'gender': 'Women', 'heel_type': 'Flats', 'occasion': 'Casual', 'type': 'Flip Flops', 'product_price': 1119.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'blue'}
                                                                                                                                                               

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-18-1060-black-casual-sandals.html
Metro
Black Casual Sandals
Men
Flats
Casual
Sandals
3990.0
Rs
6,7,8,9,10,11
UK
black,brown,navy,camel
{'product_url': 'https://www.metroshoes.com/metro-18-1060-black-casual-sandals.html', 'product_brand': 'Metro', 'product_name': 'Black Casual Sandals', 'gender': 'Men', 'heel_type': 'Flats', 'occasion': 'Casual', 'type': 'Sandals', 'product_price': 3990.0, 'currency_symbol': 'Rs', 'product_sizes': '6,7,8,9,10,11', 'size_country': 'UK', 'product_colors': 'black,brown,navy,camel'}
                                                                                                                                                   

...

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/davinchi-75-1131-black-formal-sandals.html
Davinchi
Black Formal Sandals
women
Stilettos
Formal
Sandals
3990.0
Rs
3,4,5,6,7,8
UK
black,beige
{'product_url': 'https://www.metroshoes.com/davinchi-75-1131-black-formal-sandals.html', 'product_brand': 'Davinchi', 'product_name': 'Black Formal Sandals', 'gender': 'women', 'heel_type': 'Stilettos', 'occasion': 'Formal', 'type': 'Sandals', 'product_price': 3990.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'black,beige'}
                                                                                                                                                     

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-41-3834-yellow-casual-slides.html
Metro
Yellow Casual Slides
Women
Block
Casual
Slides
1990.0
Rs
3,4,5,6,7,8
UK
black,grey,white,green,yellow,peach
{'product_url': 'https://www.metroshoes.com/metro-41-3834-yellow-casual-slides.html', 'product_brand': 'Metro', 'product_name': 'Yellow Casual Slides', 'gender': 'Women', 'heel_type': 'Block', 'occasion': 'Casual', 'type': 'Slides', 'product_price': 1990.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'black,grey,white,green,yellow,peach'}
                                                                                                                           

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-40-2437-green-casual-slippers.html
Metro
Green Casual Slippers
Women
Block
Casual
Slippers
2490.0
Rs
3,4,5,6,7,8
UK
beige,green,peach
{'product_url': 'https://www.metroshoes.com/metro-40-2437-green-casual-slippers.html', 'product_brand': 'Metro', 'product_name': 'Green Casual Slippers', 'gender': 'Women', 'heel_type': 'Block', 'occasion': 'Casual', 'type': 'Slippers', 'product_price': 2490.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'beige,green,peach'}
                                                                                                                                                       

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/metro-34-9694-black-casual-sandals.html
Metro
Black Casual Sandals
Women
Platform
Casual
Sandals
2990.0
Rs
3,4,5,6,7,8
UK
black,tan
{'product_url': 'https://www.metroshoes.com/metro-34-9694-black-casual-sandals.html', 'product_brand': 'Metro', 'product_name': 'Black Casual Sandals', 'gender': 'Women', 'heel_type': 'Platform', 'occasion': 'Casual', 'type': 'Sandals', 'product_price': 2990.0, 'currency_symbol': 'Rs', 'product_sizes': '3,4,5,6,7,8', 'size_country': 'UK', 'product_colors': 'black,tan'}
                                                                                                                                                                       

----------------------------------------------------------------------------------------------------
                                                                                                                                                                                                        
https://www.metroshoes.com/skechers-239-124905-grey-sports-walking-shoes.html
Skechers
Gray/Pink Sports Sneakers
Women
Flats
Sports
Sneakers
10999.0
Rs
6,7,8,9,10
US
grey,navy
{'product_url': 'https://www.metroshoes.com/skechers-239-124905-grey-sports-walking-shoes.html', 'product_brand': 'Skechers', 'product_name': 'Gray/Pink Sports Sneakers', 'gender': 'Women', 'heel_type': 'Flats', 'occasion': 'Sports', 'type': 'Sneakers', 'product_price': 10999.0, 'currency_symbol': 'Rs', 'product_sizes': '6,7,8,9,10', 'size_country': 'US', 'product_colors': 'grey,navy'}
                                                                                                                                     

In [10]:
df = pd.read_csv('metroshoes.csv')
df.shape
# we saw 2596 proudct counts from the website 

(2437, 12)

In [11]:
len(metroshoes.product_urls) # checking crawled products urls count

2596

In [12]:
filter_ = set(metroshoes.product_urls) # checking whether it has dublicate
print(len(filter_)) 

2437
